In [ ]:
import re
import pandas as pd

In [ ]:
reformatted_MRs = []
das = []
domain = "tv actor"
with open("/content/tv_actors_mrs_new.txt", "r") as infile:
  for index, line in enumerate(infile):
    pattern = r"(\w+)\[(.*?)\]"

    da_pattern = r'^(\w+)\('
    da_match = re.match(da_pattern, line)

    # Extract attributes and their values from the input string
    matches = re.findall(pattern, line)

    # Initialize an empty dictionary to store attribute-value pairs
    attribute_dict = {}

    # Loop through the matches and store the attribute-value pairs in the dictionary
    for match in matches:
        attribute = match[0]
        value = match[1].replace("{", "").replace("}", "").replace("'", "")
        attribute_dict[attribute] = value

    das.append(da_match.group(1))
    output_str = f"{da_match.group(1)} = yes | "
    for attribute, value in attribute_dict.items():
      output_str += f"{attribute} = {value} | "

    output_str += f"domain = {domain}"

    # Remove the trailing "|" and print the output string
    output_str = output_str.rstrip(" | ")

    reformatted_MRs.append(output_str)

In [ ]:
len(reformatted_MRs)

22542

In [ ]:
# Define regular expression pattern for matching attribute values
pattern = r"\[([^\]]*)\]"
# Define regular expression pattern for matching attribute names
attribute_name_pattern = r"([a-z_]+)\["
movie_pseudo_mrs = []

# Open input and output files
with open("/content/tv_actors_mrs_new.txt", "r") as infile:
    # Loop over input file, extracting and processing each line
    for line in infile:
        # Use regular expression to extract attribute values
        matches = re.findall(pattern, line)
        # Extract individual attribute values and format them as desired
        values = [match.replace("_", " ").title() if match else "" for match in matches]
        #print("values: ", values)
        values = [value.replace('{', '') for value in values]
        values = [value.replace('}', '') for value in values]
        # If no attribute value is found, extract the attribute name
        if not any(values):
            attribute_names = re.findall(attribute_name_pattern, line)
            values = [name.replace("_", " ").title() for name in attribute_names]
        # Join formatted values into a single string
        modified_text = " ".join(values)
        # print("modified text: ", modified_text)
        # Write modified text to txt file

        movie_pseudo_mrs.append(modified_text)

In [ ]:
len(movie_pseudo_mrs)

4256

In [ ]:
combined_data = []
for index, mr in enumerate(reformatted_MRs):
  combined_data.append((mr, das[index], movie_pseudo_mrs[index], "tv", "tv actor", 1))

In [ ]:
len(combined_data)

4256

In [ ]:
movies_data_df = pd.DataFrame(combined_data, columns=["mr", "da", "test_pseudo_utt", "domain", "type", "n_hops"])

In [ ]:
len(movies_data_df)

4256

In [ ]:
movies_data_df.head()

,mr,da,test_pseudo_utt,domain,type,n_hops
0,inform = yes | name = Jodie Comer | height = t...,inform,Jodie Comer Taller 1993,tv,tv actor,1
1,inform = yes | name = Amy Poehler | cast_membe...,inform,Amy Poehler 'The Mighty B!' Female 1971 'Star ...,tv,tv actor,1
2,inform = yes | name = Rashida Jones | place_of...,inform,Rashida Jones Los Angeles,tv,tv actor,1
3,inform = yes | name = Finn Wolfhard | date_of_...,inform,Finn Wolfhard 2002 Male Vancouver,tv,tv actor,1
4,inform = yes | name = Josh Dallas | date_of_bi...,inform,Josh Dallas 1978 Louisville Male,tv,tv actor,1


In [ ]:
num_triples = []

In [ ]:
for index, val in enumerate(movies_data_df['mr']):
    num_triples.append(val.count("|")-1)

In [ ]:
movies_data_df['num_triples'] = num_triples

In [ ]:
movies_data_df['mr'][0]

'inform = yes | name = Jodie Comer | height = taller | date_of_birth = 1993 | domain = tv actor'

In [ ]:
# wiki_attributes_mapping = {'genres': 'genre',
#                            'cast_member': 'cast member',
#                            'publication_date': 'publication date',
#                            'producer': 'producer',
#                            'screenwriter': 'screenwriter',
#                            'director': 'director',
#                            'character_role': 'character',
#                            'rating': 'review score',
#                            'MPA_rating': 'MPA film rating',
#                            'specifier': 'specifier'
#                            }

# wiki_attributes_mapping = {'genres': 'genre',
#                            'cast_member': 'cast member',
#                            'date_of_birth': 'date of birth',
#                            'producer': 'producer',
#                            'screenwriter': 'screenwriter',
#                            'director': 'director',
#                            'character_role': 'character',
#                            'rating': 'review score',
#                            'MPA_rating': 'MPA film rating',
#                            'specifier': 'specifier',
#                            'awards_received': 'awards received',
#                            'creator': 'creator'
#                            }

wiki_attributes_mapping = {'gender': 'gender',
                           'date_of_birth': 'date of birth',
                           'cast_member': 'cast member of',
                           'spouse': 'spouse',
                           'name': 'name',
                           'place_of_birth': 'place of birth',
                           'height': 'height',
                           'rating': 'rating',
                           'awards_received': 'awards received'
                          }

# wiki_attributes_mapping = {
#    "good_with_children": "good with children",
#    "good_with_other_dogs": "good with other dogs",
#    "coat_length": "coat length",
#    "good_with_strangers": "good with strangers",
#    "max_life_expectancy": "max life expectancy",
#    "max_height_male": "max height male",
#    "max_weight_male": "max weight male",
#    "max_height_female": "max height female",
#    "max_weight_female": "max weight female"
# }

In [ ]:
wikiID_list = []
def create_pseudo_mr_new(path):
  string = path
  # Split the string into key-value pairs
  pairs = string.split(" | ")
  pseudo_mr_string = ""
  request_attr = 0
  # print("pairs: ", pairs)
  name = "actor"
  for index, pair in enumerate(pairs):
      key, value = pair.split(" = ")
      if value == "yes":
        if key == "request_attribute":
          print("yea")
          request_attr = 1
        else:
          request_attr = 0
        continue
      if index == 1 and key == "name":
        name = value
        # mask = wiki_id_df["name"] == value
        # filtered_df = wiki_id_df[mask]
        # specific_column_value = filtered_df["wikiID"].values[0]
        # wikiID_list.append(specific_column_value)
        continue
      key = wiki_attributes_mapping.get(key, key)
      if request_attr == 1:
         print("in here")
         pseudo_mr_sub = f"tv actor {key}. "
         req_attr_keys.append(key)
         request_attr = 0
      elif index == len(pairs)-1:
        pseudo_mr_sub = f"{name} {key} {value}."
        # print(pseudo_mr_sub)
      else:
        pseudo_mr_sub = f"{name} {key} {value}. "
      pseudo_mr_string += pseudo_mr_sub
  return pseudo_mr_string

In [ ]:
create_pseudo_mr_new(movies_data_df['mr'][6])

'Ronald Reagan occupation film actor. Ronald Reagan height taller. Ronald Reagan date of birth 1911. Ronald Reagan cast member of Tear down this wall!. Ronald Reagan awards received Order of the White Eagle. Ronald Reagan domain actor.'

In [ ]:
movies_data_df["pseudo_mr"] = movies_data_df["mr"].apply(create_pseudo_mr_new)

yea
in here


In [ ]:
len(movies_data_df)

4256

In [ ]:
len(wikiID_list)

4635

In [ ]:
movies_data_df['mr'][0]

'inform = yes | name = George Hamilton | rating = highly rated | gender = male | place_of_birth = Memphis | height = taller | spouse = Alana Stewart | date_of_birth = 1939 | cast_member = Love at First Bite | domain = actor'

In [ ]:
movies_data_df

,mr,da,test_pseudo_utt,domain,type,n_hops,num_triples,pseudo_mr
0,inform = yes | name = Jodie Comer | height = t...,inform,Jodie Comer Taller 1993,tv,tv actor,1,3,Jodie Comer height taller. Jodie Comer date of...
1,inform = yes | name = Amy Poehler | cast_membe...,inform,Amy Poehler 'The Mighty B!' Female 1971 'Star ...,tv,tv actor,1,5,Amy Poehler cast member of The Mighty B!. Amy ...
2,inform = yes | name = Rashida Jones | place_of...,inform,Rashida Jones Los Angeles,tv,tv actor,1,2,Rashida Jones place of birth Los Angeles. Rash...
3,inform = yes | name = Finn Wolfhard | date_of_...,inform,Finn Wolfhard 2002 Male Vancouver,tv,tv actor,1,4,Finn Wolfhard date of birth 2002. Finn Wolfhar...
4,inform = yes | name = Josh Dallas | date_of_bi...,inform,Josh Dallas 1978 Louisville Male,tv,tv actor,1,4,Josh Dallas date of birth 1978. Josh Dallas pl...
...,...,...,...,...,...,...,...,...
4251,verify_attribute = yes | name = Kristen Bell |...,verify_attribute,Kristen Bell Top Rated 'Frozen',tv,tv actor,1,3,Kristen Bell rating top rated. Kristen Bell ca...
4252,verify_attribute = yes | name = Ted Danson | r...,verify_attribute,Ted Danson Top Rated Taller Male,tv,tv actor,1,4,Ted Danson rating top rated. Ted Danson height...
4253,verify_attribute = yes | name = Justin Hartley...,verify_attribute,Justin Hartley Highly Rated Knoxville,tv,tv actor,1,3,Justin Hartley rating highly rated. Justin Har...
4254,verify_attribute = yes | name = Johnny Galecki...,verify_attribute,Johnny Galecki Top Rated 'The Big Bang Theory'...,tv,tv actor,1,4,Johnny Galecki rating top rated. Johnny Galeck...


In [ ]:
movies_data_df.to_csv("tv_actor_data_for_generation_one_hop.csv")